In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, avg, lit, udf, count, sum
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas
import numpy as np
import json
import requests
import os

from data_utils import get_date_range, get_data, clear_data
from datetime import datetime

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
def scrape_language_info_with_comments(dates):
    gh_df = get_data(dates, spark)
    gh_df.createOrReplaceTempView("events")
    gh_df = spark.sql("SELECT repo.name as Repo, " + 
                      "replace(payload.comment.body, ',', '')  as Comment, " + 
                      "payload.pull_request.head.repo.language as Language, " +
                      "payload.pull_request.head.repo.forks_count as Forks_Count, " +
                      "payload.pull_request.head.repo.stargazers_count as Stargazers_count, " +
                      "payload.pull_request.head.repo.open_issues_count as Open_issues_count " +
                      "FROM events " + 
                      "WHERE type = 'PullRequestReviewCommentEvent' " + 
                      "AND payload.comment.body IS NOT NULL AND payload.pull_request.head.repo.language IS NOT NULL "+
                      "AND CHAR_LENGTH(payload.comment.body) > 0"
                     )
    
    
    with open('comments.csv', 'a+') as file:
        for row in gh_df.collect():
            d = row.asDict()
            s = "%s,%s,%s,%d,%d,%d\n" % (d["Repo"], d["Language"], d["Comment"], d["Forks_Count"], d["Stargazers_count"], d["Open_issues_count"])
            file.write(s)

    return gh_df

In [ ]:
for i in range(1, 31):
    for j in range(0, 24, 2):
        try:
            dates = get_date_range(datetime(2020, 3, i, j), datetime(2020, 3, i, j+1))
            df = scrape_language_info_with_comments(dates)
            clear_data()
        except:
            print('Error')
            continue